In [ ]:
import os
os.getcwd()
os.path.isfile("drive/MyDrive/ML_TEXT_STUFF/test_data.txt")

True

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
"""
Before using this, make sure to download from https://nlp.stanford.edu/projects/glove/ the glove.twitter.27B.zip 
and unzip it in the twitter-datasets folder !

PS: you need quite a lot of RAM :)

"""

#actually, params are from https://towardsdatascience.com/sentiment-analysis-for-text-with-deep-learning-2f0a0c6472b5
full = True
max_len = 128  # max # of words in a the tweets (result of a linear search)
dimension = 50
batch_size = 500
epochs = 10
activation_function = "relu"
#learning_rate = 0.005 # TODO : TRY BETTER ONES
path = "drive/MyDrive/ML_TEXT_STUFF/"

from IPython.core.debugger import set_trace
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import time
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
# from keras import Constant,Embedding, LSTM, Dense, Dropout,Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.initializers import Constant
from tensorflow.keras import initializers
import re
import gc
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
from datetime import datetime
import os.path

#gc.collect()
def remove_URL(text):
    url = re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"", text)

def remove_html(text):
    html = re.compile(r"<.*?>")
    return html.sub(r"", text)

def remove_emoji(string):
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        "]+",
        flags=re.UNICODE,
    )
    return emoji_pattern.sub(r"", string)

def remove_punct(text):
    table = str.maketrans("", "", string.punctuation)
    return text.translate(table)



def remove_stopwords(text):
    text = [word.lower() for word in text.split() if word.lower() not in stop]

    return " ".join(text)


# ### Embeddings

# #### Using GloVe

# https://nlp.stanford.edu/projects/glove/



def create_corpus_tk(train):
    corpus = []
    for text in train:
        words = [word.lower() for word in word_tokenize(text)]
        corpus.append(words)
    return corpus
#from stackoverflow
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]
def get_train_labels_with_test(train_percentage,full):
    path = "drive/MyDrive/ML_TEXT_STUFF/"
    #train_sentences,train_labels,test_sentences,test_labels = list(),list(),list(),list()
    if(full):
        full = "_full"
    else:
        full = ""
    train = list()
    labels = list()
    with open(path + "train_pos"+full+".txt",encoding='utf-8',errors="namereplace") as f :
        for pos_line in f:
            train.append(pos_line.replace("\n"," "))
            labels.append(1)
    train_size_pos = int(len(labels)* train_percentage)
    train_sentences = train[:train_size_pos]
    train_labels = labels[:train_size_pos]
    test_sentences = train[train_size_pos:]
    test_labels = labels[train_size_pos:]
    trained_size = len(train)
    labels = list()
    with open(path + "train_neg"+full+".txt",encoding='utf-8',errors="namereplace") as f :
        for neg_line in f:
            train.append(neg_line.replace("\n"," "))
            labels.append(0)
    train_size_neg = int((len(labels))* train_percentage)
    
    train_sentences.extend(train[trained_size:train_size_neg+trained_size])
    train_labels.extend(labels[:train_size_neg])
    test_sentences.extend(train[train_size_neg+trained_size:])
    test_labels.extend(labels[train_size_neg:])
    return train,unison_shuffled_copies(np.array(train_sentences),np.array(train_labels)),unison_shuffled_copies(np.array(test_sentences),np.array(test_labels))
# def get_train_label():
#     assert False #make sure you never use this method again
#     path = "twitter-datasets\\"
#     train = list()
#     labels = list()
#     with open(path + "train_pos_full.txt",encoding='utf-8',errors="namereplace") as f :
#         for pos_line in f:
#             train.append(pos_line)
#             labels.append(1)
#     with open(path + "train_neg_full.txt",encoding='utf-8',errors="namereplace") as f :
#         for neg_line in f:
#             train.append(neg_line)
#             labels.append(0)
#     return train,labels
def load_test_data():
    path = "drive/MyDrive/ML_TEXT_STUFF/"
    test = list()
    indices = list()
    with open(path + "test_data.txt",encoding='utf-8',errors="namereplace") as f :
        for test_line in f:
            sep = test_line.find(",")
            id_ = test_line[0:sep]
            tweet = test_line[sep+1:]
            test.append(tweet)
            indices.append(id_)
    return test,indices

plt.style.use(style="seaborn")

path_to_npy = "NUMPY_TEMPORARY_FILE.npy"
if(True or not os.path.isfile(path_to_npy) or full == False):
    total_train,(train_sentences,train_labels),(test_sentences,test_labels) = get_train_labels_with_test(0.85,full)
    corpus = create_corpus_tk(total_train) # takes approx. 10 minutes in full or <1 min in not full
    total_train = list() # let the garbage collector free some RAM
    gc.collect()
    if(False and full):
        with open(path_to_npy,"wb") as f:
            np.save(f,train_sentences)
            np.save(f,train_labels)
            np.save(f,test_sentences)
            np.save(f,test_labels)
            np.save(f,np.array(corpus))
else:
    print("No need")
    with open(path_to_npy,"rb") as f:
        train_sentences = np.load(f)
        train_labels= np.load(f)
        test_sentences = np.load(f)
        test_labels = np.load(f)
        corpus = np.load(f,allow_pickle=True)
        corpus = corpus.tolist()
    print("Test ",len(train_sentences))


print("Number of total tweets ",len(train_sentences), len(test_sentences))
num_words = len(corpus)
print(num_words,flush = True)

# 157 is average #words per tweet + 2* its standard deviation


tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(train_sentences)


train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(
    train_sequences, maxlen=max_len, truncating="post", padding="post"
)
# train_padded

test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(
    test_sequences, maxlen=max_len, padding="post", truncating="post"
)
test_sequences,train_sequences = None,None
train_sentences,test_sentences = None,None # let the garbage collector free some RAM
gc.collect()
# test_padded
word_index = tokenizer.word_index
print("Number of unique words:", len(word_index))
##################################################################################################
# test_data,indices = load_test_data()
# sequences = tokenizer.texts_to_sequences(test_data)
# padded = pad_sequences(sequences, maxlen=max_len, padding="post", truncating="post")
# print("padded is",padded,len(padded[0]),flush=True)
# assert False
###################################################################################################
print("Begin to construct embedding_dict",flush=True)

embedding_dict = {}
path = "drive/MyDrive/ML_TEXT_STUFF/"
with open(path+"glove.twitter.27B/glove.twitter.27B."+str(dimension)+"d.txt", "r",encoding="utf-8",errors="namereplace") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vectors = np.asarray(values[1:], "float32")
        embedding_dict[word] = vectors
f.close()
print("Finish to construct embedding_dict",flush=True)

num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, dimension))

for word, i in word_index.items():
    if i < num_words:
        emb_vec = embedding_dict.get(word)
        if emb_vec is not None:
            embedding_matrix[i] = emb_vec


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Number of total tweets  2125000 375000
2500000
Number of unique words: 457397
Begin to construct embedding_dict
Finish to construct embedding_dict


In [6]:
  
import os
import shutil
import numpy as np
import nltk 
nltk.download('wordnet')
import string
import gensim
import pickle
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
import random
import math


import matplotlib.pyplot as plt

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [7]:
class TEXT_MODEL(tf.keras.Model):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_matrix,
                 max_input_length,
                 embedding_dimensions=128,
                 cnn_filters=50,
                 dnn_units=512,
                 model_output_classes=2,
                 dropout_rate=0.1,
                 training=False,
                 name="text_model"):
        super(TEXT_MODEL, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocabulary_size,
                                          embedding_dimensions,
                                           embeddings_initializer=Constant(embedding_matrix),
                                          input_length=max_input_length,
                                          trainable=False
                                            )
        self.cnn_layer1 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=2,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer2 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=3,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer3 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=4,
                                        padding="valid",
                                        activation="relu")
        self.pool = layers.GlobalMaxPool1D()
        
        self.dense_1 = layers.Dense(units=dnn_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)

        self.last_dense = layers.Dense(units=1, activation="sigmoid")
        
    
    def call(self, inputs, training):
        l = self.embedding(inputs)
        l_1 = self.cnn_layer1(l) 
        l_1 = self.pool(l_1) 
        l_2 = self.cnn_layer2(l) 
        l_2 = self.pool(l_2)
        l_3 = self.cnn_layer3(l)
        l_3 = self.pool(l_3) 
        
        concatenated = tf.concat([l_1, l_2, l_3], axis=-1) # (batch_size, 3 * cnn_filters)
        concatenated = self.dense_1(concatenated)
        concatenated = self.dropout(concatenated, training)
        model_output = self.last_dense(concatenated)
        
        return model_output

In [8]:
EMB_DIM = dimension
CNN_FILTERS = 100
DNN_UNITS = 256

DROPOUT_RATE = 0.2

NB_EPOCHS = epochs

text_model = TEXT_MODEL(vocabulary_size=num_words,
                    embedding_matrix = embedding_matrix,
                    max_input_length = max_len,
                    embedding_dimensions=EMB_DIM,
                    cnn_filters=CNN_FILTERS,
                    dnn_units=DNN_UNITS,
                    model_output_classes=2,
                    dropout_rate=DROPOUT_RATE)


text_model.compile(loss="binary_crossentropy",
                    optimizer="adam",
                    metrics=["accuracy"])


text_model.fit(
    train_padded,
    train_labels,
    epochs=epochs, # TODO : INCREASE EPOCH, IT IS NOW LOW FOR TESTING PURPOSES - ~ 9 MIN / EPOCH IN NON FULL VS 4H IN FULL
    validation_data=(test_padded, test_labels),
    verbose=1,
    batch_size = batch_size,
    use_multiprocessing = True,
    
    
    #train_data, epochs=NB_EPOCHS
    
    )

Epoch 1/10
4250/4250 [==============================] - 1890s 445ms/step - loss: 0.3936 - accuracy: 0.8135 - val_loss: 0.3787 - val_accuracy: 0.8235
Epoch 2/10
4250/4250 [==============================] - 1875s 441ms/step - loss: 0.3606 - accuracy: 0.8338 - val_loss: 0.3668 - val_accuracy: 0.8294
Epoch 3/10
4250/4250 [==============================] - 1879s 442ms/step - loss: 0.3497 - accuracy: 0.8400 - val_loss: 0.3634 - val_accuracy: 0.8316
Epoch 4/10
4250/4250 [==============================] - 1879s 442ms/step - loss: 0.3427 - accuracy: 0.8437 - val_loss: 0.3646 - val_accuracy: 0.8307
Epoch 5/10
4250/4250 [==============================] - 1883s 443ms/step - loss: 0.3379 - accuracy: 0.8462 - val_loss: 0.3577 - val_accuracy: 0.8359
Epoch 6/10
4250/4250 [==============================] - 1882s 443ms/step - loss: 0.3339 - accuracy: 0.8483 - val_loss: 0.3573 - val_accuracy: 0.8362
Epoch 7/10
4250/4250 [==============================] - 1880s 442ms/step - loss: 0.3309 - accuracy: 0.8498

Hello there,
I had a similar issue with my model when using Adam or RMSprop. Combination of relu activation function, random_normal kernel initializer and SGD optimizer seems to have solved the problem for me. Good luck!


In [9]:
#sequences = tokenizer.texts_to_sequences(test.text)
train_labels,test_labels = None,None # let the garbage collector free some RAM
gc.collect()
test_data,indices = load_test_data()
sequences = tokenizer.texts_to_sequences(test_data)
padded = pad_sequences(sequences, maxlen=max_len, padding="post", truncating="post")
print("padded is",padded,flush=True)
pred = text_model.predict(padded)
pred_int = pred.round().astype("int")

print("prediction is",pred)
print("type of pred is ",type(pred))

print("assert equation ",indices[len(indices)-1],len(pred))
try:
    resFile = open("sub_GLOVE_"+str(full)+"dim"+str(dimension)+"_batch_size_"+str(batch_size)+"_epochs_"+str(epochs)+"___"+str(datetime.now()).replace(" ","__").replace(":","-")+".csv","w")
    resFile.write("Id,Prediction\n")
    for i in range(len(pred_int)):
        predicted = pred_int[i]
        if(predicted == 0):
            predicted = -1
        elif(predicted != 1):
            print("Prediction type error on ",predicted)
        resFile.write(str(indices[i])+","+str(int(predicted))+"\n")
except :
    print("Error encountered, try again")
finally:
    resFile.close()

print("Rounded prediction is ",pred_int)

padded is [[1822 3825  733 ...    0    0    0]
 [   1 9801   96 ...    0    0    0]
 [   2  274  395 ...    0    0    0]
 ...
 [  29  319    9 ...    0    0    0]
 [   1   22  215 ...    0    0    0]
 [ 406    6  345 ...    0    0    0]]
prediction is [[8.6135864e-02]
 [3.6104596e-01]
 [2.9709250e-01]
 ...
 [1.7830729e-04]
 [8.7229621e-01]
 [3.8059056e-03]]
type of pred is  <class 'numpy.ndarray'>
assert equation  10000 10000
Rounded prediction is  [[0]
 [0]
 [0]
 ...
 [0]
 [1]
 [0]]


In [12]:
print(str(text_model))